<a href="https://colab.research.google.com/github/Sandeeppasala/S4-Assignment-Solution/blob/master/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import all the needed libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)   # input 28x28x2  -> output 28x28x16 2D Convolution 
        self.bn1 = nn.BatchNorm2d(16)                 # input 28x28x16 2D Batch Normalization
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)  # input 28x28x16 -> output 28x28x16 2D Convolution
        self.bn2 = nn.BatchNorm2d(16)                 # input 28x28x16 2D Batch Normalization
        self.pool1 = nn.MaxPool2d(2, 2)               # input 28x28x16 -> output 14x14x16 Max Pooling
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1)  # input 14x14x16 -> output 14x14x16 2D Convolution
        self.bn3 = nn.BatchNorm2d(16)                 # input 14x14x16 2D Batch Normalization
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1)  # input 14x14x16 -> output 14x14x16 2D Convolution
        self.bn4 = nn.BatchNorm2d(16)                 # input 14x14x16 2D Batch Normalization
        self.pool2 = nn.MaxPool2d(2, 2)               # input 14x14x16 -> output 7x7x16 Max Pooling
        self.conv5 = nn.Conv2d(16, 16, 3)             # input 7x7x16 -> output 5x5x16 2D Convolution
        self.bn5 = nn.BatchNorm2d(16)                 # input 5x5x16 2D Batch Normalization
        self.conv6 = nn.Conv2d(16, 16, 3)             # input 5x5x16 -> output 3x3x16 2D Convolution
        self.bn6 = nn.BatchNorm2d(16)                 # input 3x3x16 2D Batch Normalization
        self.conv7 = nn.Conv2d(16, 10, 3)             # input 3x3x16 -> output 1x1x10
        self.dout = nn.Dropout(p=0.25)                 # Drop Out

#calculate Convolution apply RELU apply Batch Normalization
#Drop out is applied after maxpooling.

    def forward(self, x):
        x = self.pool1(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x)))))))
        x = self.dout(x)
        x = self.pool2(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x)))))))
        x = self.dout(x)
        x = self.bn6(F.relu(self.conv6(self.bn5(F.relu(self.conv5(x))))))
        x = self.conv7(x)
        x = self.dout(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)     # Softmax to return the final value.

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

#After reducing the channels to 16 the model parameters were reduced to 13K

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 16, 28, 28]           2,320
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
           Dropout-6           [-1, 16, 14, 14]               0
            Conv2d-7           [-1, 16, 14, 14]           2,320
       BatchNorm2d-8           [-1, 16, 14, 14]              32
            Conv2d-9           [-1, 16, 14, 14]           2,320
      BatchNorm2d-10           [-1, 16, 14, 14]              32
        MaxPool2d-11             [-1, 16, 7, 7]               0
          Dropout-12             [-1, 16, 7, 7]               0
           Conv2d-13             [-1, 16, 5, 5]           2,320
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#Epoch = 20  Accuracy acheived = 99.48%

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0503, Accuracy: 9854/10000 (98.54%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0353, Accuracy: 9889/10000 (98.89%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0176, Accuracy: 9939/10000 (99.39%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0170, Accuracy: 9949/10000 (99.49%)



loss=0.4067247807979584 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 34.65it/s]



Test set: Average loss: 0.0200, Accuracy: 9944/10000 (99.44%)

